In [26]:
from langchain import LLMChain, PromptTemplate
from langchain_openai import ChatOpenAI
import requests
import os


os.environ["OPENAI_API_KEY"] = <place your tokens here>



# Define tools as functions

In [27]:
def get_pokemon_data(pokemon_name: str):
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_name.lower()}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": "Pokémon not found"}

def extract_pokemon_stats(pokemon_data: dict):
    if "error" in pokemon_data:
        return pokemon_data["error"]
    
    stats = pokemon_data["stats"]
    return {stat["stat"]["name"]: stat["base_stat"] for stat in stats}

def format_pokemon_info(pokemon_data: dict, stats: dict):
    if "error" in pokemon_data:
        return pokemon_data["error"]
    
    info = f"Pokémon: {pokemon_data['name'].capitalize()}\n"
    info += f"Height: {pokemon_data['height']}\n"
    info += f"Weight: {pokemon_data['weight']}\n"
    info += "Stats:\n"
    for stat_name, stat_value in stats.items():
        info += f"  {stat_name.capitalize()}: {stat_value}\n"
    
    return info


# Create the Pokémon Info Chain


In [28]:
class PokemonInfoChain:
    def __init__(self):
        self.get_pokemon_data = get_pokemon_data
        self.extract_pokemon_stats = extract_pokemon_stats
        self.format_pokemon_info = format_pokemon_info

    def run(self, pokemon_name: str):
        # Step 1: Fetch Pokémon data
        pokemon_data = self.get_pokemon_data(pokemon_name)
        
        # Step 2: Extract Pokémon stats
        pokemon_stats = self.extract_pokemon_stats(pokemon_data)
        
        # Step 3: Format Pokémon information
        formatted_info = self.format_pokemon_info(pokemon_data, pokemon_stats)
        
        return formatted_info


# Define the Combined Prompt Template


In [29]:
# Initialize the LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


# Define a combined prompt template
combined_prompt_template = PromptTemplate(
    input_variables=["prompt"],
    template="""
    You are an intelligent assistant. Your task is to determine if a given prompt is asking about a Pokémon.
    If it is, extract the Pokémon name and fetch its data. If it is not, answer the general question.
    
    Here is the prompt: {prompt}

    Step 1: Determine if the prompt is asking about a Pokémon. 
    Step 2: If it is, extract the Pokémon name and use the Pokémon Info Chain to fetch its data. 
    Step 3: If it is not, answer the general question using the general LLM chain.

    example output:
    "This is a Pokémon-related prompt. The Pokémon name is 'pikachu'."
    or 
    "This is a general question. The answer is 'I am a general answer.'"
    """
)


In [37]:
class IntelligentAssistant:
    def __init__(self):
        self.llm = llm
        self.combined_prompt_template = combined_prompt_template
        self.pokemon_info_chain = PokemonInfoChain()

    def handle_prompt(self, prompt: str):

        combined_chain = self.combined_prompt_template | llm

        response = combined_chain.invoke(prompt)
  
        response_text = response.content

        if "pokémon" in response_text.lower():
            pokemon_name = response_text.split(" ")[-1]
            return self.pokemon_info_chain.run(pokemon_name)
        else:
            return response

# Example usage
assistant = IntelligentAssistant()

prompt = "Who is Pikachu?"
print(assistant.handle_prompt(prompt))

prompt = "Tell me about Charizard."
print(assistant.handle_prompt(prompt))

prompt = "What is the weather today?"
print(assistant.handle_prompt(prompt))


Pokémon not found
Pokémon: Charizard
Height: 17
Weight: 905
Stats:
  Hp: 78
  Attack: 84
  Defense: 78
  Special-attack: 109
  Special-defense: 85
  Speed: 100

Pokémon not found
